# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [33]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 
spark = SparkSession.builder.appName("data-wrangling-quiz").getOrCreate()
path = "data/sparkify_log_small.json"

# Question 1

Which page did user id "" (empty string) NOT visit?

In [9]:
# TODO: write your code to answer question 1
user_log = spark.read.json(path)
user_log.printSchema()
# user_log.describe().show()
user_log.select("page").where(user_log.userId=="").dropDuplicates().show()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



In [35]:
# filter for users with blank user id
blank_pages = user_log.filter(user_log.userId == '') \
    .select(col('page') \
    .alias('blank_pages')) \
    .dropDuplicates()

# get a list of possible pages that could be visited
all_pages = user_log.select('page').dropDuplicates()

# find values in all_pages that are not in blank_pages
# these are the pages that the blank user did not go to
for row in set(all_pages.collect()) - set(blank_pages.collect()):
    print(row.page)

Downgrade
Error
Save Settings
NextSong
Submit Upgrade
Logout
Upgrade
Settings
Submit Downgrade


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [15]:
# TODO: use this space to explore the behavior of the user with an empty string
user_log.select(["userId","ts","firstname","page"]).where(user_log.userId=="").groupBy("page").count().orderBy("count").collect()

[Row(page='Help', count=8),
 Row(page='About', count=15),
 Row(page='Login', count=126),
 Row(page='Home', count=187)]

# Question 3

How many female users do we have in the data set?

In [21]:
# TODO: write your code to answer question 3
user_log.select(["userId", "gender"]).where(user_log.gender=="F").dropDuplicates().count()

462

In [37]:
user_log.filter(user_log.gender == 'F') \
    .select('userId', 'gender') \
    .dropDuplicates() \
    .count()

462

# Question 4

How many songs were played from the most played artist?

In [30]:
# TODO: write your code to answer question 4
# groupedData object must agg to show
user_log.groupBy("artist").count().orderBy("count", ascending=False).take(5)

[Row(artist=None, count=1653),
 Row(artist='Coldplay', count=83),
 Row(artist='Kings Of Leon', count=69),
 Row(artist='Florence + The Machine', count=52),
 Row(artist='BjÃ\x83Â¶rk', count=46)]

In [34]:
user_log.filter(user_log.page == 'NextSong') \
    .select('Artist') \
    .groupBy('Artist') \
    .agg({'Artist':'count'}) \
    .withColumnRenamed('count(Artist)', 'Artistcount') \
    .sort(desc('Artistcount')) \
    .show(1)

+--------+-----------+
|  Artist|Artistcount|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [38]:
# TODO: write your code to answer question 5
function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

